dcf model version b

In [30]:
class FloatList(list):

    def __init__(self, *args):
        super().__init__(args)

    def __format__(self, format_spec):
        return f'[{", ".join(f"{i:{format_spec}}" for i in self)}]'

class FloatList_v1(list):

    def __init__(self, *args):
        super().__init__(args)

    def __format__(self, format_spec):
        return f'[{", ".join(f"{i:{format_spec}}" for i in self)}]'

fl = FloatList(0.1, 0.33, 0.632)
print(f"{fl:.2f}")  # [0.10, 0.33, 0.63]
fl = FloatList_v1(0.1, 0.33, 0.632)
print(f"{fl:.2f}")  # [0.10, 0.33, 0.63]


[0.10, 0.33, 0.63]
[0.10, 0.33, 0.63]


input

In [44]:
import numpy as np
import numpy_financial as npf
# general input
period = 20
interest_rate = 0.05
cap_cost = [-100.0,-30.0,-20.0]
const_period = len(cap_cost)
loan_ratio = 0.8
gw =125*16/1000
gwh = gw * 0.3 * 8760


# opex input
gwh = gw*0.3*8760   # 125 turbines each 16 mw running at 30% capacity factor
strike_price_base_year = 46*1000  # per GWH, from £46 per MWh
base_year = 2012
completion_year = 2029
cpi = 2.3/100

strike_price_comp_year = strike_price_base_year * (1+cpi)**(completion_year - base_year)

opex_per_GW = 75   # 75 £m per GW
base_year_opex = 2029
completion_year = 2029
opex_cpi = 2.3/100

opex_per_GW_comp_year = opex_per_GW * (1+opex_cpi)**(completion_year - base_year_opex)

print(f'gwh: {gwh:.2f}, strike price at completion: {strike_price_comp_year:.2f}, \
    opex per GW at completion: {opex_per_GW_comp_year}')

# finance input
depreciation_period = 10  # both for wind farm and sub stations
debt_repayment_period = 5


gwh: 5256.00, strike price at completion: 67708.56,     opex per GW at completion: 75.0


calculate capex

In [45]:
loan = [i * loan_ratio for i in cap_cost]
cap_int =[]
loan_bal = []
prev_loan_bal = 0
for i in range(len(loan)):
    
    loan_bal_avg = prev_loan_bal + loan[i]/2
    
    int = loan_bal_avg * interest_rate
    loan[i] += int
    loan_balance = prev_loan_bal + loan[i]
    cap_int.append(int)
    loan_bal.append(loan_balance)
    prev_loan_bal = loan_balance
capex = [x + y for x, y in zip(cap_cost, cap_int)]
equity = [x - y for x, y in zip(capex, loan)]
capex_sum = - sum(capex)
loan_sum = - sum(loan)
equity_sum = - sum(equity)
wf = FloatList_v1(loan)
print(f"loan draw down: {loan}, loan balance: {loan_bal}")
print(f'capex: {capex}, including capital costs: {cap_cost} and capitalized interest: {cap_int}')
print(f'capex: {capex}, financed by equity: {equity} and loan: {loan}\
    \ntotal capex:{capex_sum:.1f}, financed by equity: {equity_sum:.2f} and loan: {loan_sum:.2f}')
#for i in range(period):  cap_cost.append(0)
#for i in range(period):  loan.append(0)

loan draw down: [-82.0, -28.7, -21.935000000000002], loan balance: [-82.0, -110.7, -132.635]
capex: [-102.0, -34.7, -25.935000000000002], including capital costs: [-100.0, -30.0, -20.0] and capitalized interest: [-2.0, -4.7, -5.9350000000000005]
capex: [-102.0, -34.7, -25.935000000000002], financed by equity: [-20.0, -6.0000000000000036, -4.0] and loan: [-82.0, -28.7, -21.935000000000002]    
total capex:162.6, financed by equity: 30.00 and loan: 132.63


revenue calculation

In [46]:
annual_revenue = strike_price_comp_year * gwh /1000000  # in £ millions 
revenue = []
for i in range(period):  revenue.append(annual_revenue * (1+cpi)**i)
print(f'revenue in £ millions (nominal): {revenue}')
annual_opex = opex_per_GW_comp_year * gw
opex = []
for i in range(period):  opex.append(annual_opex * (1+opex_cpi)**i)
print(f'revenue in £ millions (nominal): {revenue} \n opex in £millions (nominal): {opex}')


revenue in £ millions (nominal): [355.87621373316006, 364.0613666490227, 372.4347780819502, 381.000777977835, 389.76379587132516, 398.7283631763656, 407.89911552942203, 417.2807951865987, 426.8782534758904, 436.69645330583586, 446.74047173187006, 457.01550258170295, 467.5268591410821, 478.279976901327, 489.2804163700574, 500.5338659465687, 512.0461448633397, 523.8232061951965, 535.871139937686, 548.1961761562527]
revenue in £ millions (nominal): [355.87621373316006, 364.0613666490227, 372.4347780819502, 381.000777977835, 389.76379587132516, 398.7283631763656, 407.89911552942203, 417.2807951865987, 426.8782534758904, 436.69645330583586, 446.74047173187006, 457.01550258170295, 467.5268591410821, 478.279976901327, 489.2804163700574, 500.5338659465687, 512.0461448633397, 523.8232061951965, 535.871139937686, 548.1961761562527] 
 opex in £millions (nominal): [150.0, 153.45, 156.97934999999998, 160.58987504999996, 164.28344217614995, 168.06196134620137, 171.927386457164, 175.88171634567877, 1

depreciation calculation

In [47]:
annual_depreciation = capex_sum / depreciation_period
depreciation = []
for i in range(period):  depreciation.append(annual_depreciation)

debt payment calculation

In [49]:
annual_debt_repayment = loan_sum / debt_repayment_period
op_int =[]
# loan_bal = []
for i in range(debt_repayment_period):  loan.append(annual_debt_repayment)

prev_loan_bal = loan_bal[-1]
for i in range(debt_repayment_period):
    j = i + const_period
    loan_bal_avg = prev_loan_bal - loan[j]/2
    
    int = loan_bal_avg * interest_rate
   
    loan_balance = prev_loan_bal + loan[j]
    op_int.append(int)
    loan_bal.append(loan_balance)
    prev_loan_bal = loan_balance
print(f"loan: {loan}, \nloan balance: {loan_bal}")
print(f'op interest: {op_int}')


loan: [-82.0, -28.7, -21.935000000000002, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997, 26.526999999999997], 
loan balance: [-82.0, -110.7, -132.635, -159.16199999999998, -185.68899999999996, -212.21599999999995, -238.74299999999994, -265.2699999999999, -238.74299999999994, -212.21599999999995, -185.68899999999996, -159.16199999999998, -132.635]
op interest: [-13.926674999999998, -12.600324999999998, -11.273974999999998, -9.947624999999999, -8.621274999999999]


profit and loss

In [ ]:
ebit = capex = [x - y for x, y in zip(revenue, opex)]